In [ ]:
!pip install "transformers[sentencepiece]" "transformers[torch]" evaluate datasets rouge_score rouge numba numpy pandas arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.7 MB/s eta 0:

In [ ]:
import torch
import datasets
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoTokenizer
from rouge import Rouge
import numpy as np
import evaluate
from numba import cuda
import re
import json
import pandas as pd

In [ ]:
from arabert.preprocess import ArabertPreprocessor

In [ ]:
!gdown https://drive.google.com/uc?id=1-VWQqbbWWAcEqkyW1GkKJojpn2Si1kV2&export=download
!gdown https://huggingface.co/datasets/csebuetnlp/xlsum/resolve/main/data/arabic_XLSum_v2.0.tar.bz2
!tar -xvf /content/arabic_XLSum_v2.0.tar.bz2

Downloading...
From: https://drive.google.com/uc?id=1-VWQqbbWWAcEqkyW1GkKJojpn2Si1kV2
To: /content/arabic.pkl
100% 119M/119M [00:00<00:00, 212MB/s]
Downloading...
From: https://huggingface.co/datasets/csebuetnlp/xlsum/resolve/main/data/arabic_XLSum_v2.0.tar.bz2
To: /content/arabic_XLSum_v2.0.tar.bz2
100% 44.9M/44.9M [00:00<00:00, 436MB/s]
./
./arabic_test.jsonl
./arabic_val.jsonl
./arabic_train.jsonl


In [ ]:
import pickle
import os

# Wikilingua Dataset

with open('/content/arabic.pkl', 'rb') as f:
  arabic_docs=pickle.load(f)

X=[]  # Data
Y=[]  # Labels
for link in arabic_docs.values():
  for part in link.values():
    X.append(part['document'])
    Y.append(part['summary'])

In [ ]:
from datasets import load_dataset
checkpoint = "malmarjeh/t5-arabic-text-summarization"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
preprocessor = ArabertPreprocessor(model_name="")

In [ ]:
# texts=[]
# summaries = []

x_valid = []
y_valid = []
# Labeled validation dataset
with open('labeled_validation_dataset.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        if data is None: continue
        text=data['paragraph']
        summary = data['summary']
        x_valid.append(text)
        y_valid.append(summary)

# X = texts + X
# Y = summaries + Y

with open('/content/arabic_train.jsonl', 'r') as f:
  for line in f:
    data = json.loads(line)
    X.append(data['text'])
    Y.append(data['summary'])

with open('/content/arabic_test.jsonl', 'r') as f:
  for line in f:
    data = json.loads(line)
    X.append(data['text'])
    Y.append(data['summary'])

with open('/content/arabic_val.jsonl', 'r') as f:
  for line in f:
    data = json.loads(line)
    X.append(data['text'])
    Y.append(data['summary'])

df1 = pd.DataFrame(list(zip(X, Y)),
               columns =['paragraph', 'summary'])
df2 = pd.DataFrame(list(zip(x_valid,y_valid)),
                  columns = ['paragraph', 'summary'])

dataset = datasets.Dataset.from_pandas(df1, split='train')
dataset_test = datasets.Dataset.from_pandas(df2,split = "test")

In [ ]:
# dataset=dataset.train_test_split(test_size=0.1, seed=42, shuffle=True)
dataset

Dataset({
    features: ['paragraph', 'summary'],
    num_rows: 76098
})

In [ ]:
dataset_test

Dataset({
    features: ['paragraph', 'summary'],
    num_rows: 154
})

In [ ]:
# dataset['train'][0]

{'paragraph': 'وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب عرض الكتاب الرابع ، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67 وأنها ارتاحت لاعتقادها بأن هناك وقتا طويلا وطويلا جدا قبل أن يفيق العرب من صدمة 67، وكيف أن القوات الجوية للجمهورية العربية المتحدة قد فاجأتها بعد شهر واحد من نهاية حرب 67 بهجوم جوى عنيف على مواقعها فى سيناء وكان هذا إعلانا عن بداية حرب من نوع جديد هى حرب الاستنزاف التى استمرت حتى تم وقف إطلاق النار بين الطرفين فى 8 أغسطس 1970، ثم وفاة عبدالناصر وتولى أنور السادات حكم مصر واستعداده للحرب . ويتعرض الكاتب أيضا وبصورة سريعة لفلسطين والأردن وسوريا قبل أن ينتقل إلى الكتاب الخامس عن حرب أكتوبر ، حيث يعرض الخطط والاستعدادات المصرية ثم الاستعدادات الإسرائيلية ثم يبدأ بعرض وقائع الحرب بداية من الضربة الجوية وانهيار خط بارليف واختراقه ، ويتوقف الكاتب عند يوم 8 أكتوبر ، ويقول : إن هذا اليوم كان اسوأ هزيمة فى تاريخ الجيش الإسرائيلى ثم ينتقل بنا المؤلف إلى الجبهة السورية ثم يعود ثانية إلى يوميات الحرب حتى 7 9 أكتوبر إلى 9 13 أكتوبر ثم 14 أكتوبر ، ثم يعرض للثغرة أو ما عرف بعمل

In [ ]:
def tokenize_function(examples):
  inputs = [ preprocessor.preprocess(doc) for doc in examples['paragraph']]
  model_inputs = tokenizer(inputs, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], padding=True, max_length=128, truncation=True, return_tensors="pt")
  model_inputs['labels'] = labels['input_ids'].cuda()
  model_inputs['input_ids'] = model_inputs['input_ids'].cuda()
  print(model_inputs['input_ids'])

  return model_inputs

In [ ]:
def tokenize_function_xlsum(examples):
  inputs = [ preprocessor.preprocess(doc) for doc in examples['text']]
  model_inputs = tokenizer(inputs, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], padding=True, max_length=128, truncation=True, return_tensors="pt")
  model_inputs['labels'] = labels['input_ids'].cuda()
  model_inputs['input_ids'] = model_inputs['input_ids'].cuda()
  print(model_inputs['input_ids'])

  return model_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/76098 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


tensor([[  215,  2165, 54318,  ...,   493, 49664,     1],
        [32686,  1857,   836,  ..., 13270,    60,     1],
        [    3,    16, 68957,  ...,    40,   481,     1],
        ...,
        [   21,   345,  2270,  ...,     0,     0,     0],
        [  168,   234,   777,  ...,    57, 33878,     1],
        [  168, 14847,    40,  ...,     0,     0,     0]], device='cuda:0')
tensor([[    8,   589,     6,  ...,     0,     0,     0],
        [ 7659, 18716, 48162,  ...,     6,   676,     1],
        [  615,   279,   506,  ...,     0,     0,     0],
        ...,
        [30825,    31,   640,  ...,    22,    77,     1],
        [ 1278,  1131,     6,  ...,  3041,    13,     1],
        [ 5963,   194, 42177,  ...,  2721,  3065,     1]], device='cuda:0')
tensor([[ 6364,     6,  3482,  ...,   114,    24,     1],
        [ 7897, 28831,    31,  ...,     8,  1378,     1],
        [  279,  7236, 58661,  ..., 30365,     3,     1],
        ...,
        [    8,  2986,     3,  ..., 18520, 37787,     1

In [ ]:
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

tensor([[ 5549,  2824,     6,  ...,     0,     0,     0],
        [  352, 13788, 16436,  ...,     0,     0,     0],
        [ 1812,  3277,  2365,  ...,     0,     0,     0],
        ...,
        [44363,    24,   257,  ...,     0,     0,     0],
        [  352, 25558, 21027,  ...,     0,     0,     0],
        [  416,   147, 43766,  ...,     0,     0,     0]], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors='pt')

In [ ]:
def calculate_rouge(predictions, references):
  scores = Rouge().get_scores(predictions, references, avg=True)
  scores = {"rouge1": round(scores['rouge-1']['f'], 4), "rouge2": round(scores['rouge-2']['f'], 4), "rougeL" : round(scores['rouge-l']['f'], 4)}
  return scores

In [ ]:
t1 = 'هذا مجرد نص تجريبي'
t2 = 'نص تجريبي'
t3 = 'هذا النص للتجربة'
t4 = 'النص الجديد'

calculate_rouge([t1, t3], [t2, t4])

{'rouge1': 0.5333, 'rouge2': 0.25, 'rougeL': 0.5333}

In [ ]:
model_out = []
from transformers import pipeline

def calc_metrics(eval_pred):
  global model_out
  model_out = eval_pred

  torch.device('cpu')
  data = dataset['test']['paragraph']
  labels = dataset['test']['summary']
  pipe = pipeline('summarization', model=model, tokenizer=tokenizer, device=0)
  predictions = pipe(data, length_penalty=1.6, num_beams=8, min_length=100, max_length=200, repetition_penalty=4.0)
  predictions = [item['summary_text'] for item in predictions]

  result = calculate_rouge(predictions, labels)
  predictions_lens = [np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in predictions]

  result["gen_len"]=np.mean(predictions_lens)
  torch.device('cuda')
  return {k:round(v,4) for k,v in result.items()}

In [ ]:
x, y = model_out
y = np.where(y != -100, y, tokenizer.pad_token_id)
(tokenizer.batch_decode(x, skip_special_tokens=True)[0])

In [ ]:
tokenizer.decode(y[0], skip_special_tokens=True)

In [ ]:
temp = tokenizer(dataset['test']['paragraph'][0], truncation=True, max_length=512, padding='max_length', return_tensors='pt')

In [ ]:
model.generate(temp['input_ids'].cuda(), max_length=128)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [ ]:
# torch.device / CUDA Setup
import torch

if torch.cuda.is_available():
    torch_device = torch.device('cuda')

    # Set this to True to make your output immediately reproducible
    # Note: https://pytorch.org/docs/stable/notes/randomness.html
    torch.backends.cudnn.deterministic = False

    # Disable 'benchmark' mode: Set this False if you want to measure running times more fairly
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = True

    # Faster Host to GPU copies with page-locked memory
    use_pin_memory = True

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False


CUDA Version: 11.8
cuDNN Version: 8700
CUDA Device Name: NVIDIA A100-SXM4-40GB
CUDA Capabilities: (8, 0)


In [ ]:
tokenized_dataset

Dataset({
    features: ['paragraph', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 76098
})

In [ ]:
tokenized_dataset_test

Dataset({
    features: ['paragraph', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 154
})

In [ ]:
import huggingface_hub as hf
hf.notebook_login()

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="AlQalam-finetuned-mmj-withXlsumValid",
    evaluation_strategy="epoch",
    learning_rate=0.0005,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

Cloning https://huggingface.co/Grinta-king/AlQalam-finetuned-mmj-withXlsumValid into local empty directory.


In [ ]:
# torch.cuda.empty_cache()
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.479500,4.134305
2,1.328800,4.045451
3,1.219000,3.955738
4,1.153100,3.982416


TrainOutput(global_step=12684, training_loss=1.3380746002341626, metrics={'train_runtime': 13310.2188, 'train_samples_per_second': 22.869, 'train_steps_per_second': 0.953, 'total_flos': 1.8536191022333952e+17, 'train_loss': 1.3380746002341626, 'epoch': 4.0})

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
import shutil
# shutil.rmtree('/content/AlQalam-finetuned-mmj-withXlsumValid/checkpoint-11500')
# shutil.rmtree('/content/AlQalam-finetuned-mmj-withXlsumValid/checkpoint-12500')
# shutil.rmtree('/content/AlQalam-finetuned-mmj-withXlsumValid/checkpoint-12000')
shutil.rmtree('/content/AlQalam-finetuned-mmj-withXlsumValid/*')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/lib/python3.10/shutil.py:715 in rmtree                                                      │
│                                                                                                  │
│    712 │   │   try:                                                                              │
│    713 │   │   │   orig_st = os.lstat(path)                                                      │
│    714 │   │   except Exception:                                                                 │
│ ❱  715 │   │   │   onerror(os.lstat, path, sys.exc_info())                                       │
│    716 │   │   │   return                                                                        │
│    717 │   │   try:                                                                              │
│    718 │   │   │   fd = os.open(path, os.O_RDONLY)                                               │
│                                                                                                  │
│ /usr/lib/python3.10/shutil.py:713 in rmtree                                                      │
│                                                                                                  │
│    710 │   │   # Note: To guard against symlink races, we use the standard                       │
│    711 │   │   # lstat()/open()/fstat() trick.                                                   │
│    712 │   │   try:                                                                              │
│ ❱  713 │   │   │   orig_st = os.lstat(path)                                                      │
│    714 │   │   except Exception:                                                                 │
│    715 │   │   │   onerror(os.lstat, path, sys.exc_info())                                       │
│    716 │   │   │   return                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '/content/AlQalam/checkpoint-7000'

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.05G [00:00<?, ?B/s]

Upload file runs/Jul06_15-50-19_4707850c3432/events.out.tfevents.1688658639.4707850c3432.437.0:   0%|         …

To https://huggingface.co/Grinta-king/AlQalam-finetuned-mmj-withXlsumValid
   6f768d1..ba6f046  main -> main

   6f768d1..ba6f046  main -> main

To https://huggingface.co/Grinta-king/AlQalam-finetuned-mmj-withXlsumValid
   ba6f046..63cf5f3  main -> main

   ba6f046..63cf5f3  main -> main



'https://huggingface.co/Grinta-king/AlQalam-finetuned-mmj-withXlsumValid/commit/ba6f04620d444feca0e9369ccbae5cd8b21b26a0'

In [ ]:
trainer.push_to_hub()

Cloning https://huggingface.co/omar-atef/AlQalam into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/1.05G [00:00<?, ?B/s]

Upload file spiece.model:   0%|          | 1.00/2.32M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/4.00k [00:00<?, ?B/s]

To https://huggingface.co/omar-atef/AlQalam
   40bf6c1..73a6398  main -> main

   40bf6c1..73a6398  main -> main

To https://huggingface.co/omar-atef/AlQalam
   73a6398..e244306  main -> main

   73a6398..e244306  main -> main



'https://huggingface.co/omar-atef/AlQalam/commit/73a6398b67626be7e06c92c37040e48ce86d6180'

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()
# model = AutoModelForSeq2SeqLM.from_pretrained('./AraT5-finetuned-on-validation-dataset',local_files_only=True)
# tokenizer = AutoTokenizer.from_pretrained('./AraT5-finetuned-on-validation-dataset',local_files_only=True)

1051

In [ ]:
len(summaries)

154

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('Grinta-king/AlQalam')
tokenizer = AutoTokenizer.from_pretrained('Grinta-king/AlQalam')

In [ ]:
from transformers import pipeline
summ = pipeline('summarization', model=model, tokenizer=tokenizer, device=0)

In [ ]:
import torch
torch.device('cuda:0')

device(type='cuda', index=0)

In [ ]:
summ(texts[:2], length_penalty=1.6, num_beams=8, min_length=100, max_length=200, repetition_penalty=4.0)[0]['summary_text']

'بدأ الكاتب عرض الكتاب الرابع تحت عنوان من الكارثة إلى التحدى، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67، وكيف بدأت حرب الاستنزاف، ثم يبدأ بالعرض للخطط والاستعدادات المصرية، ثم الاستعدادات الإسرائيلية، ثم ينتقل بنا إلى الجبهة السورية، ثم يعود ثانية إلى يوميات الحرب، ثم يعرض الكاتب وقائع الحرب، ثم سينتقل بنا إلى الجبهة السورية، ثم وينتقل بنا إلى الجبهة السورية، ثم يدخل الكاتب المعركة الخاصة بالاستيلاء على مدينة السويس من 23 أكتوبر إلى 14 أكتوبر، ثم تبدأ الحملة الخاصة بالاستيلاء على مدينة السويس من 23 أكتوبر إلى 24 أكتوبر، ثم يبدأ الكاتب بعرض وقائع الحرب الأولى في مصر 9 أكتوبر حتى تم إعفاء الفريق النقر على مدينة السويس من 23 أكتوبر إلى'

In [ ]:
predictions = summ(texts, length_penalty=1.6, num_beams=8, min_length=100, max_length=200, repetition_penalty=4.0)
labels = summaries
predictions = [item['summary_text'] for item in predictions]

Rouge().get_scores(predictions, labels, avg=True)

{'rouge-1': {'r': 0.5927127259996762,
  'p': 0.6628499720754315,
  'f': 0.6199618998893417},
 'rouge-2': {'r': 0.4734581395589518,
  'p': 0.5287174878020624,
  'f': 0.49466762549841714},
 'rouge-l': {'r': 0.5376076301409698,
  'p': 0.6036185164410782,
  'f': 0.563432433783391}}

In [ ]:
predictions[0]

'نشرت صحيفة الفاينانشال تايمز كتابا جديدا بعنوان "الحرب على إسرائيل" يتناول فيه أحداث الحرب العالمية الثانية التي وقعت في أكتوبر/تشرين الأول عام 1967، وذلك بعد مرور أكثر من نصف قرن على إعلان الجيش الإسرائيلي انسحابه من الأراضي المصرية وإحتلالها للأراضي الفلسطينية المحتلة خلال حرب 67 والتي أدت إلى الإطاحة بالرئيس المصري الراحل عبد الفتاح السيسي ورئيس الوزراء السابق محمد حسني مبارك عن حكم الجمهورية العربية المتحدة قبل خمس سنوات. ومن أهم الكتب التي تناولتها جريدة التايمز منذ بدء الحملة الجوية الإسرائيلية على قطاع غزة يوم 6 أكتوبر/ تشرين الأول الماضي.'

In [ ]:
summaries[0]

'يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الكارثة إلى التحدى؛ حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67، وكيف بدأت حرب الاستنزاف، ثم يتكلم عن وفاة عبدالناصر، وتولى أنور السادات حكم مصر، ويعرض الكاتب للخطط والاستعدادات المصرية، ثم الاستعدادات الإسرائيلية، ثم يبدأ بعرض وقائع الحرب، ويتوقف الكاتب عند يوم 8 أكتوبر، ويقول: إن هذا اليوم كان أسوأ هزيمة فى تاريخ الجيش الإسرائيلى، ثم ينتقل بنا إلى الجبهة السورية، ثم يعود ثانية إلى يوميات الحرب من  9 أكتوبر إلى 13 و 14 أكتوبر، والعمليات النهائية فى سوريا 14 إلى 23 أكتوبر، ثم يعرض الكاتب المعركة الخاصة بالاستيلاء على مدينة السويس من 23 أكتوبر إلى 25 أكتوبر، ثم تطورات هذه المعركة. '

In [ ]:
texts[5]

'قال السفير المصري لدي الكويت ياسر عاطف إن 6 أكتوبر يوم الفخر لمصر والأمة العربية جميعا " وتحضرني هنا كلمة خالدة للرئيس الراحل أنور السادات حينما قال أن كفاحنا كان كفاحا من اجل الحق والحرية " . وأضاف عاطف في تصريحات للصحفيين خلال احتفال مكتب الاتصال العسكري بالسفارة المصرية بالكويت بذكري مرور 43 عام على نصر أكتوبر المجيد " واعتقد أن كفاحنا حتى اليوم مازال كفاحا من اجل الحق والحرية , فكل عام نحتفل بهذا الانتصار المجيد ونحي هذه الذكري الخالدة التي تحمل الكثير من الدروس وقصص البطولة . وعن المشاركة الكويتية والعربية في حرب أكتوبر قال السفير ياسر عاطف " التعاون العربي مع مصر بدءا من حرب الاستنزاف وجسد إحساس الأمة العربية بعمق الروابط التي تجمعها مع مصر ووقوف العرب صفا واحدا لافتا إلى انه كان في الطليعة العربية الضباط والجنود الكويتيين . وقال " ويحضر معنا الاحتفال اليوم العميد عبد الله المقلد احد الأبطال الكويتيين الذين شاركوا مع القوات المصرية منذ حرب الاستنزاف ابتداء من سنة 1968 والتي توجت بانتصارات أكتوبر المجيدة وهي صورة من صور التضامن والتكامل العربي بشكل واضح " . وأضاف عاطف قائلا " ثم 

In [ ]:
predictions = []
i=1

for text in texts:
  words = get_len(text)
  if words >= 200:
    min_length = int(0.25*words)
    max_length = int(0.65*words)
  elif words >= 100:
    min_length = int(0.35*words)
    max_length = int(0.75*words)
  else:
    min_length = int(0.65*words)
    max_length = words

  # min_length=100
  # max_length=200
  pred = summ(text, length_penalty=1.6, num_beams=3, min_length=min_length, max_length=max_length, repetition_penalty=5.0)[0]['summary_text']
  predictions.append(pred)
  print(i)
  i+=1